# Trabalho 1 - Inteligência Artificial

## Imports necessários

In [47]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.dummy import DummyClassifier
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

## Base de dados

### Matrícula: 2015100346
Dessa forma, devido ao final da matrícula ser 6, a base de dados será composta pelos 10 descritores de Fourier e os 7 descritores de Hu.

In [48]:
# Leitura dos dados
df = pd.read_csv('https://raw.githubusercontent.com/VitorBonella/PL-Dataset/main/dataset.csv',sep=";") 

# Transformando a coluna id no índice da tabela
df.set_index('id', inplace=True)

# Lista de descritores
FOURIER = ['df01', 'df02', 'df03', 'df04','df05', 'df06', 'df07', 'df08', 'df09', 'df10']
HU = ['i1', 'i2', 'i3', 'i4','i5', 'i6', 'i7']
HARALICK = ['probmax', 'energia', 'entropia', 'contraste','homogeneidade', 'correlacao']

# Descritores que serão usados nesse trabalho
dataset = df[FOURIER + HU] 

# Transformação dos dados de string para float devido ao uso da vírgula ao invés do ponto
dataset = dataset.apply(lambda x: x.str.replace(',', '.').astype(float), axis=1)

# Criação das classes baseada no tipo da lâmpada e na potência
classes = df['tipo_lampada'].str.replace(" ", "") + df['potencia'].astype(str) 

# Adiciona a classe ao data frame da base de dados
# dataset['classe'] = df['tipo_lampada'].str.replace(" ", "") + df['potencia'].astype(str) 

# Define a base de dados e as classes target
dataset_X = dataset
dataset_Y = classes

# from sklearn import datasets
# dataset = datasets.load_breast_cancer()
# dataset_X = dataset.data
# dataset_Y = dataset.target

print(classes)
dataset.head(2)


id
355    metalica400
356    metalica400
357    metalica400
358    metalica400
359    metalica400
          ...     
656    metalica250
657    metalica250
658    metalica250
659    metalica250
660    metalica250
Length: 297, dtype: object


,df01,df02,df03,df04,df05,df06,df07,df08,df09,df10,i1,i2,i3,i4,i5,i6,i7
id,,,,,,,,,,,,,,,,,
355,0.087961,0.054059,0.025210,0.027471,0.012834,0.008374,0.004592,0.005163,0.004747,0.003075,0.168084,0.000615,0.000131,5.717751e-05,3.808159e-09,-1.417337e-06,3.153370e-09
356,0.018333,0.064192,0.021335,0.028935,0.012263,0.011230,0.003566,0.005862,0.003746,0.003779,0.164507,0.001087,0.000043,1.011576e-08,-5.395931e-15,-6.859661e-11,3.907229e-15


# Cálculo dos resultados

In [49]:
def classification_report(scores):
    print(f'\nMédia: {scores.mean():.5f}, Desvio Padrão: {scores.std():.5f}')

    inf, sup = stats.norm.interval(0.95, loc=scores.mean(), 
                               scale=scores.std()/np.sqrt(len(scores)))
    
    print(f'Intervalo de confiança (95%): [{inf:.5f},{sup:.5f}]')

# ZeroR (ZR)

In [50]:
zR = DummyClassifier(random_state=11)

pipeline = Pipeline([('transformer', StandardScaler()), ('estimator', zR)])

rkf = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=36851234)

scores_zR = cross_val_score(pipeline, dataset_X, dataset_Y, scoring='accuracy', cv = rkf)

print(scores_zR)

classification_report(scores_zR)

[0.16666667 0.13333333 0.16666667 0.16666667 0.16666667 0.16666667
 0.16666667 0.17241379 0.17241379 0.17241379 0.16666667 0.13333333
 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.17241379
 0.17241379 0.17241379 0.16666667 0.13333333 0.16666667 0.16666667
 0.16666667 0.16666667 0.16666667 0.17241379 0.17241379 0.17241379]

Média: 0.16506, Desvio Padrão: 0.01088
Intervalo de confiança (95%): [0.16116,0.16895]


# Bagging (BA)

In [51]:
grade = {'estimator__n_estimators':[3, 9, 15, 21]}

# TODO
# Talvez usar um estimador diferente no final do ensemble
# Opções: Decision Tree, Random Forest, K-Nearest Neighbors (KNN), Support Vector Machines (SVM)
bg = BaggingClassifier(estimator=GaussianNB(), random_state=11)

pipeline = Pipeline([('transformer', StandardScaler()), ('estimator', bg)])

gs = GridSearchCV(estimator=pipeline, param_grid=grade, scoring='accuracy', cv=4)

rkf = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=36851234)

scores_BA = cross_val_score(gs, dataset_X, dataset_Y, scoring='accuracy', cv = rkf)

print(scores_BA)

classification_report(scores_BA)

[0.36666667 0.3        0.2        0.33333333 0.3        0.33333333
 0.4        0.27586207 0.4137931  0.48275862 0.26666667 0.2
 0.3        0.33333333 0.43333333 0.56666667 0.33333333 0.34482759
 0.34482759 0.37931034 0.33333333 0.5        0.23333333 0.4
 0.26666667 0.26666667 0.4        0.4137931  0.27586207 0.31034483]

Média: 0.34360, Desvio Padrão: 0.08377
Intervalo de confiança (95%): [0.31363,0.37358]


# AdaBoost (AB)

In [52]:
grade = {'estimator__n_estimators':[3, 9, 15, 21]}

# TODO
# Talvez usar um estimador diferente no final do ensemble
# Opções: Decision Tree, Random Forest, K-Nearest Neighbors (KNN), Support Vector Machines (SVM)
adb = AdaBoostClassifier(estimator=GaussianNB(), random_state=11)

pipeline = Pipeline([('transformer', StandardScaler()), ('estimator', adb)])

gs = GridSearchCV(estimator=pipeline, param_grid=grade, scoring='accuracy', cv=4)

rkf = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=36851234)

scores_AB = cross_val_score(gs, dataset_X, dataset_Y, scoring='accuracy', cv = rkf)

print(scores_AB)

classification_report(scores_AB)

[0.46666667 0.16666667 0.33333333 0.33333333 0.23333333 0.2
 0.26666667 0.37931034 0.20689655 0.48275862 0.3        0.23333333
 0.4        0.26666667 0.46666667 0.26666667 0.2        0.51724138
 0.24137931 0.31034483 0.23333333 0.33333333 0.33333333 0.3
 0.33333333 0.2        0.36666667 0.27586207 0.34482759 0.4137931 ]

Média: 0.31352, Desvio Padrão: 0.09113
Intervalo de confiança (95%): [0.28091,0.34614]


# RandomForest (RF)

In [53]:
grade = {'randomForest__n_estimators': [3, 9, 15, 21]}

rF = RandomForestClassifier(random_state=11)

pipeline = Pipeline([('transformer', StandardScaler()), ('randomForest', rF)])

gs = GridSearchCV(estimator=pipeline, param_grid=grade, scoring='accuracy', cv = 4)

rkf = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=36851234)

scores_RF = cross_val_score(gs, dataset_X, dataset_Y, scoring='accuracy', cv = rkf)

print(scores_RF)

classification_report(scores_RF)


[0.5        0.36666667 0.36666667 0.6        0.53333333 0.43333333
 0.4        0.55172414 0.55172414 0.75862069 0.53333333 0.4
 0.63333333 0.43333333 0.5        0.66666667 0.4        0.65517241
 0.62068966 0.34482759 0.66666667 0.6        0.3        0.46666667
 0.56666667 0.5        0.46666667 0.62068966 0.55172414 0.48275862]

Média: 0.51571, Desvio Padrão: 0.10981
Intervalo de confiança (95%): [0.47641,0.55500]


# Heterogeneous Pooling (HP)

In [157]:
from sklearn.utils import resample
from sklearn.utils.validation import check_X_y
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from collections import Counter
import numpy as np
from sklearn.base import BaseEstimator

class HeterogeneousPoolingClassifier(BaseEstimator):
    def __init__(self, n_samples):
        super().__init__()
        self.n_samples = n_samples
        
        # Lista com os classificadores treinados
        self.classifiers = []

        # Lista de classes ordenada de forma decrescente pela quantidade de aparições
        self.target_frequency_desc = []

    def fit(self, x_train, y_train):
        x_train, y_train = check_X_y(x_train, y_train)
        self.target_frequency_desc = self._get_target_frequency_desc(y_train)

        for i in range(self.n_samples):
            # A primeira iteração usa a base original para treinamento
            if i == 0:
                current_x_train, current_y_train = x_train, y_train
            else:
                # Base diferente da original usando resample
                current_x_train, current_y_train = resample(x_train, y_train, replace=True, random_state=i-1)

            dt_classifier = DecisionTreeClassifier(random_state=11)
            dt_classifier.fit(current_x_train, current_y_train)
            self.classifiers.append(dt_classifier)

            nb_classifier = GaussianNB()
            nb_classifier.fit(current_x_train, current_y_train)
            self.classifiers.append(nb_classifier)

            knn_classifier = KNeighborsClassifier()
            knn_classifier.fit(current_x_train, current_y_train)
            self.classifiers.append(knn_classifier)

    def predict(self, x_test):
        predictions = []
        
        # Faz a predição usando cada um dos classificadores treinados e guarda o resultado obtido
        for classifier in self.classifiers:
            print('classifier predict')
            # print(classifier.predict(x_test))
            predictions.append(classifier.predict(x_test))

        # print('original')
        # print(predictions)

        # Transpor o array faz com que cada linha da matriz sejam todas as predições de um único item
        predictions = np.transpose(predictions)

        # print('transposto')
        # print(predictions)

        # As classes mais frequente dentre as predições de cada item
        predictions = [self._most_common_prediction(prediction) for prediction in predictions]

        print('só as mais comuns')
        print(predictions)


        # for prediction in most_common_predictions:
        #     # Caso haja um empate entre as classes mais votadas, 
        #     # escolhe a que mais aparece no conjunto de teste
        #     if len(prediction) > 1:
        #         for target in self.target_frequency_desc:
        #             if target in prediction:
        #                 prediction = [target]
                
                 
            
        final_predictions = []

        return np.array(final_predictions)

    def _get_target_frequency_desc(self, y_train):
        class_counts = Counter(y_train)
        sorted_classes = sorted(class_counts, key=class_counts.get, reverse=True)
        return np.array(sorted_classes)

    def _most_common_prediction(self, item_predictions):
        # Conta a frequência de cada classe na lista de predições daquele item
        class_counts = Counter(item_predictions)

        # Determina a contagem máxima de ocorrências de uma classe
        max_count = max(class_counts.values())

        # Obtém todas as classes que possuem a contagem máxima
        most_common = [predicted_class for predicted_class, count in class_counts.items() if count == max_count]

        return most_common



In [158]:
hp = HeterogeneousPoolingClassifier(2)

pipeline = Pipeline([('transformer', StandardScaler()), ('estimator', hp)])

rkf = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=36851234)

scores_HP = cross_val_score(pipeline, dataset_X, dataset_Y, scoring='accuracy', cv = rkf)

print(scores_HP)

classification_report(scores_HP)

classifier predict
classifier predict
classifier predict
classifier predict
classifier predict
classifier predict
só as mais comuns
[['sodio400'], ['metalica400'], ['metalica250'], ['sodio150'], ['metalica250'], ['metalica400'], ['metalica400'], ['sodio150'], ['sodio250'], ['metalica250'], ['sodio100'], ['sodio100'], ['mercurio125'], ['mercurio125'], ['sodio100'], ['sodio100'], ['sodio100', 'mercurio125'], ['sodio400', 'metalica250'], ['metalica250'], ['mercurio125'], ['sodio100'], ['metalica250'], ['metalica250'], ['metalica250'], ['metalica150'], ['sodio100'], ['metalica250'], ['metalica250'], ['metalica250'], ['metalica250']]
classifier predict
classifier predict
classifier predict
classifier predict
classifier predict
classifier predict
só as mais comuns
[['sodio70', 'sodio100'], ['sodio250'], ['metalica250'], ['sodio400'], ['sodio150'], ['metalica250', 'sodio150'], ['sodio400', 'sodio100'], ['metalica250'], ['metalica250'], ['mercurio125'], ['sodio100'], ['mercurio125', 'sodio400'

C:\Users\elcin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py:794: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\elcin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_scorer.py", line 115, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\elcin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

classifier predict
classifier predict
classifier predict
classifier predict
classifier predict
classifier predict
só as mais comuns
[['sodio100'], ['metalica250'], ['sodio250', 'metalica400'], ['metalica400'], ['sodio400'], ['metalica250', 'sodio250'], ['sodio70', 'sodio100'], ['metalica150'], ['metalica250'], ['metalica400'], ['sodio100'], ['sodio100'], ['sodio100'], ['sodio100'], ['sodio400'], ['sodio100'], ['sodio400', 'metalica250', 'sodio100'], ['mercurio125'], ['mercurio125'], ['metalica250'], ['sodio100'], ['metalica400'], ['metalica250', 'metalica400'], ['sodio400', 'metalica250'], ['metalica150'], ['metalica150'], ['metalica250'], ['metalica250'], ['metalica250']]
classifier predict
classifier predict
classifier predict
classifier predict
classifier predict
classifier predict
só as mais comuns
[['metalica400'], ['metalica400'], ['metalica400'], ['metalica400', 'sodio250'], ['metalica250'], ['sodio400'], ['metalica250'], ['sodio150'], ['metalica250'], ['metalica400'], ['sodio15

C:\Users\elcin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py:794: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\elcin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_scorer.py", line 115, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\elcin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

classifier predict
classifier predict
classifier predict
classifier predict
classifier predict
classifier predict
só as mais comuns
[['sodio400'], ['sodio250'], ['sodio150'], ['metalica250'], ['sodio150'], ['sodio150'], ['sodio150'], ['metalica250'], ['metalica150', 'metalica250'], ['metalica250'], ['sodio70', 'sodio100'], ['metalica250'], ['sodio100'], ['sodio70'], ['sodio70'], ['sodio70'], ['metalica250'], ['mercurio125'], ['mercurio125'], ['metalica150', 'metalica250'], ['metalica250'], ['metalica250'], ['metalica250'], ['metalica400'], ['sodio100'], ['metalica250', 'sodio150', 'sodio250'], ['metalica250'], ['metalica150'], ['metalica250'], ['metalica250']]
classifier predict
classifier predict
classifier predict
classifier predict
classifier predict
classifier predict
só as mais comuns
[['metalica400'], ['sodio100', 'sodio400'], ['metalica250'], ['metalica250'], ['sodio250'], ['sodio250'], ['sodio250'], ['sodio250', 'metalica250'], ['metalica250'], ['sodio250'], ['sodio100', 'sodio

C:\Users\elcin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py:794: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\elcin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_scorer.py", line 115, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\elcin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

só as mais comuns
[['metalica400'], ['metalica250'], ['sodio100'], ['sodio400'], ['metalica250'], ['metalica400'], ['sodio250'], ['metalica400'], ['metalica250'], ['metalica150'], ['metalica250'], ['mercurio125'], ['sodio150'], ['metalica250'], ['sodio100'], ['sodio70', 'mercurio125', 'sodio100'], ['sodio100'], ['sodio100'], ['sodio400', 'sodio100'], ['metalica250', 'sodio400'], ['mercurio125'], ['mercurio125'], ['metalica150', 'metalica250'], ['metalica400'], ['sodio400'], ['sodio400', 'mercurio125'], ['metalica150'], ['metalica250'], ['metalica250'], ['sodio150', 'metalica250']]
classifier predict
classifier predict
classifier predict
classifier predict
classifier predict
classifier predict
só as mais comuns
[['metalica400'], ['sodio250'], ['sodio250', 'metalica250', 'metalica400'], ['sodio100'], ['metalica250'], ['metalica250'], ['metalica400'], ['sodio250', 'metalica250'], ['sodio250'], ['metalica400', 'metalica150'], ['metalica250'], ['sodio150'], ['metalica250'], ['sodio100'], ['

C:\Users\elcin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py:794: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\elcin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_scorer.py", line 115, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\elcin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

### Hp

O pseudo código a seguir mostra como o HP é obtido a partir de uma base de dados de treino:

- Obter e armazenar a ordenação das classes de acordo com a ocorrência nos exemplos na
base de treino (ordenar decrescentemente da mais frequente para a menos frequente)
- Para cada um dos n_samples faça
    - Se for a primeira iteração então
        - Usar a base original para treino dos classificadores
    - Senão
        - Montar uma base de treino de mesmo tamanho da original coletando aleatoriamente exemplos da base original com reposição
    - Fim-se
    - Treinar os classificadores NN, NB, DT na base de treino corrente e incluí-los no combinado de classificadores
- Fim-para

### Classificação final

O pseudo código seguinte mostra como o combinado HP é usado para classificar um exemplo
da base de dados de teste:

- Para cada um dos classificadores individuais do combinado faça
    - Obter a classificação do exemplo usando o classificador individual e armazenar a classe selecionada
- Fim-para
- Contar quantas vezes cada classe foi selecionada e obter a(s) mais votada(s)
- Se mais de uma classe for a mais votada então
    - Retornar a classe mais votada mais frequente na base de treino dentre as que empataram
- Senão
    - Retornar a classe mais votada
- Fim-se

In [57]:
classification_report(scores_zR)
classification_report(scores_BA)
classification_report(scores_AB)
classification_report(scores_RF)


Média: 0.16506, Desvio Padrão: 0.01088
Intervalo de confiança (95%): [0.16116,0.16895]

Média: 0.34360, Desvio Padrão: 0.08377
Intervalo de confiança (95%): [0.31363,0.37358]

Média: 0.31352, Desvio Padrão: 0.09113
Intervalo de confiança (95%): [0.28091,0.34614]

Média: 0.51571, Desvio Padrão: 0.10981
Intervalo de confiança (95%): [0.47641,0.55500]
